<a href="https://colab.research.google.com/github/monkeyasho/Speech-Diarizer/blob/main/voice2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null
!pip install pydub
!pip install -q git+https://github.com/huggingface/neuralcoref.git

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
confection 0.1.4 requires srsly<3.0.0,>=2.4.0, but you have srsly 1.0.7 which is incompatible.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 2.3.9 which is incompatible.


In [ ]:
from google.colab import files
import whisper
import datetime
import subprocess
import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
from pyannote.audio import Audio
from pyannote.core import Segment
import wave
import contextlib
from sklearn.cluster import AgglomerativeClustering
import numpy as np
from pydub import AudioSegment
import io
import requests
import re
import spacy
#import neuralcoref
#from spacy.pipeline.textcat import Config, single_label_cnn_config, single_label_bow_config
from textblob import TextBlob


/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
/usr/local/lib/python3.10/dist-packages/torch_audiomentations/utils/io.py:27: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [ ]:
# upload audio file
uploaded = files.upload()
path = next(iter(uploaded))


Saving Understanding African Politics.mp3 to Understanding African Politics.mp3


In [ ]:
# Language and model size settings
language = 'English' #@param ['any', 'English']
model_size = 'small' #@param ['tiny', 'base', 'small', 'medium', 'large']
model_name = model_size
if language == 'English' and model_size != 'large':
  model_name += '.en'

In [ ]:
# Load the pre-trained speaker embedding model
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda")
)

hyperparams.yaml:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

In [ ]:
# Load the Whisper ASR model
model = whisper.load_model(model_size)

100%|████████████████████████████████████████| 461M/461M [00:04<00:00, 106MiB/s]


In [ ]:
# Transcribe the audio
result = model.transcribe(path)
segments = result["segments"]

In [ ]:
# Check if the audio is stereo and convert it to mono if needed
#inplace conversion
audio = AudioSegment.from_file(path)
if audio.channels > 1:
    # Convert stereo to mono
    audio = audio.set_channels(1)
    # Save the mono audio to the same file
    audio.export(path, format="wav")
else:
    audio.export(path, format="wav")


In [ ]:
# Get the audio duration
with contextlib.closing(wave.open(path,'r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)


In [ ]:
audio = Audio()

# Function to get speaker embedding for a segment
def segment_embedding(segment):
    start = segment["start"]
    end = min(duration, segment["end"])
    clip = Segment(start, end)
    waveform, sample_rate = audio.crop(path, clip)
    return embedding_model(waveform[None])

In [ ]:
# Get embeddings for each segment
embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
    embeddings[i] = segment_embedding(segment)
# Handle NaN values in embeddings
embeddings = np.nan_to_num(embeddings)

In [ ]:
 # Perform clustering without specifying the number of speakers
clustering = AgglomerativeClustering(2).fit(embeddings) #works better if number of speakers are specified
labels = clustering.labels_

# Assign labels to segments
for i in range(len(segments)):
  segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

In [ ]:
# Function to format time in seconds to datetime
def time(secs):
    return datetime.timedelta(seconds=round(secs))

# Write the transcribed text along with speaker labels and timestamps to a file
f = open("transcript.txt", "w")

for (i, segment) in enumerate(segments):
  if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
    f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
  f.write(segment["text"][1:] + ' ')
f.close()

In [ ]:

#replace speaker labels
def replace_speaker_labels(input_file, output_file):
    with open(input_file, 'r') as f:
        content = f.read()

    #regular expressions to replace speaker labels
    content = re.sub(r'SPEAKER 1', 'Support Agent', content)
    content = re.sub(r'SPEAKER [2-9]', 'Customer', content)
    content = re.sub(r'SPEAKER [1-9][0-9]+', 'Customer', content)


    with open(output_file, 'w') as f:
        f.write(content)

replace_speaker_labels('transcript.txt', 'transcript.txt')


In [ ]:
'''
nlp = spacy.load("en_core_web_sm")

# Censor names and personal details
def censor_names_and_personal_details(text):
    # Identify and replace names with a generic label
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            text = text.replace(ent.text, "REDACTED_NAME")

    # Add more patterns for censoring personal details if needed
    # For example, you can look for patterns like phone numbers, email addresses, etc.

    return text
'''

'\nnlp = spacy.load("en_core_web_sm")\n\n# Censor names and personal details\ndef censor_names_and_personal_details(text):\n    # Identify and replace names with a generic label\n    doc = nlp(text)\n    for ent in doc.ents:\n        if ent.label_ == "PERSON":\n            text = text.replace(ent.text, "REDACTED_NAME")\n\n    # Add more patterns for censoring personal details if needed\n    # For example, you can look for patterns like phone numbers, email addresses, etc.\n\n    return text\n'

In [ ]:
'''
# Extract summary from transcript
def extract_summary(transcript):
    # Process the transcript text
    doc = nlp(transcript)


    # Extract key sentences for the summary
    summary_sentences = sorted(doc.sents, key=lambda x: x._.coref_clusters[0].rank)[:3]

    # Combine selected sentences into a summary
    summary = " ".join(str(sentence) for sentence in summary_sentences)

    return summary
'''

In [ ]:
'''
# Replace 'transcript.txt' with your file name or provide the actual transcript text
with open('transcript.txt', 'r') as f:
    transcript_text = f.read()

# Censor names and personal details in the transcript
censored_transcript = censor_names_and_personal_details(transcript_text)

# Extract summary from the censored transcript
summary = extract_summary(censored_transcript)

print("Censored Transcript:")
print(censored_transcript)
print("\nSummary:")
print(summary)
'''

In [ ]:
!pip install -U spacy spacy-transformers
!python -m spacy download en_core_web_trf

import spacy
'''
def censor_personal_details(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)

    censored_text = text
    for ent in doc.ents:
        if ent.label_ in ["PERSON", "PHONE", "EMAIL", "MISC"]:
            censored_text = censored_text.replace(ent.text, "***")

    return censored_text

# Example usage
transcript = """
John Doe's phone number is 123-456-7890,
and his email is john.doe@example.com.
Please contact Jane at jane@example.com regarding the account with ID: 12345.
"""

censored_transcript = censor_personal_details(transcript)
print(censored_transcript)
'''

In [ ]:
!pip install transformers
!pip install spacy

import spacy
from transformers import pipeline

# Load transformer-based NER model
nlp_transformers = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

def censor_personal_details(text):
    # Perform NER using the transformer-based model
    entities = nlp_transformers(text)

    censored_text = text

    # Censor identified entities
    for entity in entities:
        censored_text = censored_text.replace(entity['word'], '***')

    return censored_text

# Example usage
transcript = """
John Doe's phone number is 123-456-7890,
and his email is john.doe@example.com.
Please contact Jane at jane@example.com regarding the account with ID: 12345.
"""

censored_transcript = censor_personal_details(transcript)
print(censored_transcript)


In [ ]:
import spacy
import re

def censor_personal_details(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)

    censored_text = []
    for token in doc:+
        # Censor PERSON entities
        if token.ent_type_ == "PERSON":
            censored_text.append('****')
        # Censor phone numbers using regular expression
        elif re.match(r'\b\d{3}[-.\s]?\d{3}[-.\s]?\d{4}\b', token.text):
            censored_text.append('***-***-****')
        # Censor email addresses using regular expression
        elif re.match(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', token.text):
            censored_text.append('***@***.***')
        # Censor account ID
        elif re.match(r'\b\d{5}\b', token.text):
            censored_text.append('*****')
        else:
            censored_text.append(token.text)

    return ' '.join(censored_text).k5

# Example usage
transcript = """
John Doe's phone number is 123-456-7890,
and his email is john.doe@example.com.
Please contact Jane at jane@example.com regarding the account with ID: 12345.
"""

censored_transcript = censor_personal_details(transcript)
print(censored_transcript)



 **** **** **** phone number is 123 - 456 - 7890 , 
 and his email is **** . 
 Please contact Jane at ***@***.*** regarding the account with ID : ***** . 



In [ ]:
!pip install bert-extractive-summarizer
from summarizer import Summarizer

def generate_summary(file_path, ratio=0.01):

    with open(file_path, 'r') as file:
        content = file.read()

    summarizer = Summarizer()
    summary = summarizer(content, ratio=ratio)
    return summary


In [ ]:

# Example usage
file_path = '/content/transcript.txt'
summary = generate_summary(file_path)
print(summary)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Customer 0:00:00
China has been the single most deleterious force to ASEAN's economies. The first part of this extension is I'm going to explain how the CCP retains power under the status quo in China, and the first thing that they explain here is that they just create control through systems of state capitalism and through systems of gradual liberalization, which is obviously only possible through massive control, massive growth economically, which you obviously lack the funds to do when you are not a superpower on outside the house, so they're not able to do that. This is different from what O says because the US only intervenes because it cares about China's presence in the region. I first tell you that the alternative, like significant economic power, even if all of Guff's frame is correct, is not that ASEAN gets the economic benefits that exist in the world. And therefore, all trade with Asia, then if the U.S. intervention happens, happens with India and not ASEAN to a significant

In [ ]:
from transformers import WhisperForConditionalGeneration
model.config.suppress_tokens =[]

